# Μηχανική Μάθηση ~ Εργασία 1

### Θοδωρής Τσιρπάνης (`dai19090`)

---

## Προετοιμασία

Εισάγουμε τις απαραίτητες βιβλιοθήκες.

Αρκετός κώδικας είναι βασισμένος στον κώδικα των εργαστηρίων του μαθήματος

In [ ]:
from sklearn import linear_model
import numpy as np
np.set_printoptions(precision=3)
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

## Φόρτωση των δεδομένων

Για την φόρτωση των δεδομένων πρέπει πρώτα το αρχείο `abalone.data` να ανέβει στο notebook.

Πετάμε την πρώτη στήλη (το φύλο) και τοποθετούμε την τελευταία (τον στόχο) σε ξεχωριστό πίνακα.

Στο τέλος δημιουργούμε πίνακες του NumPy για να αποθηκεύσουμε τα δεδομένα μας, και εμφανίζουμε στατιστικά θέσης και διασποράς.

### Συμπεράσματα από τα δεδομένα

Επειδή τα στατιστικά των στηλών του dataset μας είναι παρόμοια σε τάξη μεγέθους, δεν υπάρχει λόγος να κανονικοποιηθούν.

In [ ]:
import csv

data_X = []
data_y = []
with open('abalone.data') as myfile:
  data = csv.reader(myfile, delimiter=',') 
  for dataline in data: 
    data_X.append(dataline[1:-1])
    data_y.append(dataline[-1])

data_X = np.array(data_X).astype(float)
data_y = np.array(data_y).astype(float)

print("Data loaded.")
print(f"Dimensions: {len(data_X)} * {len(data_X[0])}")
print(f"Mean of each column: {data_X.mean(axis = 0)}")
print(f"Standard deviation of each column: {data_X.std(axis = 0)}")

Data loaded.
Dimensions: 4177 * 7
Mean of each column: [0.524 0.408 0.14  0.829 0.359 0.181 0.239]
Standard deviation of each column: [0.12  0.099 0.042 0.49  0.222 0.11  0.139]


Διαχωρίζουμε τα σύνολα εκπαίδευσης και ελέγχου.

In [ ]:
from sklearn.model_selection import train_test_split

data_X_train, data_X_test, data_y_train, data_y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=286)

Ορίζουμε κάποιες βοηθητικές συναρτήσεις για εκπαίδευση μοντέλων, υπολογισμό του ρυθμού επιτυχίας τους, και εμφάνιση των επιδόσεών τους.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

def train(model):
  model.fit(data_X_train, data_y_train)

def success_rate(pred, test, N):
  return np.count_nonzero(np.less(np.abs(pred - test), N)) / len(pred)

def evaluate(model):
  data_y_test_pred = model.predict(data_X_test)
  data_y_train_pred = model.predict(data_X_train)
  display(Markdown(f"**Test set success rate**: {success_rate(data_y_test_pred, data_y_test, 1):.2%}<br>" +
f"**Mean squared error**: {mean_squared_error(data_y_test, data_y_test_pred):.2f}<br>" +
f"**Coefficient of determination**: {r2_score(data_y_test, data_y_test_pred):.2f}<br>" +
f"**Cross-val explained variance**: {explained_variance_score(data_y_train, data_y_train_pred):.2f}<br>" +
f"**Test set explained variance**: {explained_variance_score(data_y_test, data_y_test_pred):.2f}"))

## Απλή Γραμμική Παλινδρόμηση

In [ ]:
linearRegr = linear_model.LinearRegression()

train(linearRegr)
display(Markdown("### Simple Linear Regression"))
display(Markdown(f"**Coefficients**: {linearRegr.coef_}"))
evaluate(linearRegr)

### Simple Linear Regression

**Coefficients**: [ -1.191  13.445  10.503   9.964 -21.054  -9.991   7.369]

**Test set success rate**: 45.69%<br>**Mean squared error**: 4.42<br>**Coefficient of determination**: 0.54<br>**Cross-val explained variance**: 0.52<br>**Test set explained variance**: 0.54

## Regularization

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
           'alpha': np.logspace(-6, 4, 20)
         }
print(f"alphas: {params['alpha']}")

def trainCV(model):
  grid = GridSearchCV(estimator=model, cv = 10, param_grid=params)
  train(grid)
  display(Markdown(f"**Best lambda**: {grid.best_params_['alpha']:.3f}<br>"))
  return grid

ridge = linear_model.Ridge()
display(Markdown("### Ridge Regression"))
ridgeModel = trainCV(ridge)
display(Markdown(f"**Coefficients**: {ridgeModel.best_estimator_.coef_}"))
evaluate(ridgeModel)

lasso = linear_model.Lasso(tol=0.1)
display(Markdown("### LASSO Regression"))
lassoModel = trainCV(lasso)
display(Markdown(f"**Coefficients**: {lassoModel.best_estimator_.coef_}"))
evaluate(lassoModel)

elasticNet = linear_model.ElasticNet(tol=0.1)
display(Markdown("### ElasticNet Regression"))
elasticNetModel = trainCV(elasticNet)
display(Markdown(f"**Coefficients**: {elasticNetModel.best_estimator_.coef_}"))
evaluate(elasticNetModel)

alphas: [1.000e-06 3.360e-06 1.129e-05 3.793e-05 1.274e-04 4.281e-04 1.438e-03
 4.833e-03 1.624e-02 5.456e-02 1.833e-01 6.158e-01 2.069e+00 6.952e+00
 2.336e+01 7.848e+01 2.637e+02 8.859e+02 2.976e+03 1.000e+04]


### Ridge Regression

**Best lambda**: 0.616<br>

**Coefficients**: [  2.115   8.957   8.463   8.101 -18.895  -7.155   9.332]

**Test set success rate**: 46.29%<br>**Mean squared error**: 4.43<br>**Coefficient of determination**: 0.54<br>**Cross-val explained variance**: 0.52<br>**Test set explained variance**: 0.54

...

**Test set success rate**: 46.05%<br>**Mean squared error**: 4.42<br>**Coefficient of determination**: 0.54<br>**Cross-val explained variance**: 0.52<br>**Test set explained variance**: 0.54

### ElasticNet Regression

**Best lambda**: 0.000<br>

**Coefficients**: [  2.317   8.587   8.021   7.709 -18.437  -6.468   9.68 ]

**Test set success rate**: 46.41%<br>**Mean squared error**: 4.43<br>**Coefficient of determination**: 0.54<br>**Cross-val explained variance**: 0.52<br>**Test set explained variance**: 0.54

## Μη γραμμική παλινδρόμηση

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knRegr = KNeighborsRegressor()
params = {
  'n_neighbors': range(1,10),
  'weights': ['uniform', 'distance']
}

display(Markdown("### k-Nearest Neighbours Regression"))
grid = GridSearchCV(estimator=knRegr, cv = 10, param_grid=params) 
train(grid)
data_y_test_pred = grid.predict(data_X_test)
display(Markdown(
f"**Best score**: {grid.best_score_}<br>" +
f"**Best k**: {grid.best_params_['n_neighbors']:.3f}<br>" +
f"**Best weighting strategy**: {grid.best_params_['weights']}<br>" +
f"**Test set success rate**: {success_rate(data_y_test_pred, data_y_test, 1):.2%}<br>" +
f"**Test set explained variance**: {explained_variance_score(data_y_test, data_y_test_pred):.2f}"))

### k-Nearest Neighbours Regression

**Best score**: 0.5396870200457393<br>**Best k**: 9.000<br>**Best weighting strategy**: distance<br>**Test set success rate**: 50.00%<br>**Test set explained variance**: 0.55

## Συμπεράσματα

* Παρά την χρήση regularization, όλα τα μοντέλα γραμμικής παλινδρόμησης είχαν παρόμοιες επιδόσεις.
* Τα μοντέλα LASSO και ElasticNet φαίνεται να έχουν σχεδόν μηδενικό $λ$. Δεν ξέρω πώς να το ερμηνεύσω αυτό - το μοντέλο αποφάσισε ότι δεν χρειάζεται regularization αυτού του τύπου; Αλλά πάλι οι συντελεστές φαίνονται αρκετά διαφορετικοί. Μήπως εμφανίζεται μόνο το $λ_1$ του μοντέλου ElasticNet;
* Και τα τρία μοντέλα γραμμικής παλινδρόμησης συμφωνούν ότι υπάρχει μια ισχυρή αντίστροφη συσχέτιση μεταξύ του βάρους της σάρκας των abalones (της πέμπτης στήλης του dataset) και του αριθμού των δακτυλίων τους.
* Και τα τρία μοντέλα γραμμικής παλινδρόμησης συμφωνούν ότι η ασθενέστερη συσχέτιση μεταξύ των δεδομένων εισόδου και του στόχου συναντάται στην πρώτη στήλη του dataset (το μήκος), με το μοντέλο LASSO να της δίνει συντελεστή ίσο με το μηδέν απορρίπτοντάς την τελείως.
* Το μοντέλο των $k$ κοντινότερων γειτόνων είχε λίγο καλύτερη επίδοση από τα μοντέλα γραμμικής παλινδρόμησης, αλλά πάλι ο ρυθμός επιτυχίας αγγίζει το 50%, υπονοώντας ότι η σχέση μεταξύ των δεδομένων εισόδου και του στόχου είναι πιο σύνθετη.
* Δεν ξέρω πώς να ερμηνεύσω την υψηλή τιμή του $k$ - υπάρχει μεγάλη ποικιλομορφία στις τιμές του στόχου μεταξύ κοντινών σημείων δεδομένων και το μοντέλο αποφάσισε να κοιτάξει μακρύτερα;
* Δοκίμασα να "ταΐσω" τα δεδομένα στο εργαλείο [ML.NET Model Builder](https://dotnet.microsoft.com/en-us/apps/machinelearning-ai/ml-dotnet/model-builder) της Microsoft, και μετά από δέκα δευτερόλεπτα εκπαίδευσης και εβδομήντα διαφορετικά μοντέλα που δοκίμασε, κατέληξε σε ένα μοντέλο που το ονομάζει `LbfgsPoissonRegressionRegression` και πετυχαίνει συντελεστή $r^2$ ίσο με $0.52$, παρόμοιο με τα δικά μας μοντέλα γραμμικής παλινδρόμησης, επαληθεύοντας ότι οι επιδόσεις των μοντέλων μας δεν οφείλονται σε κάποια κακή υλοποίηση από μέρους μας.